# 1. Import libraries

In [1]:
import requests
from pandas import json_normalize
import numpy as np 
import pandas as pd

# Importing the cleaning functions:


# 2. Import data

In [2]:
smart = pd.read_csv("../data/clean_smart.csv",encoding = "utf-8")

In [3]:
smart

,Unnamed: 0,City,Country,Smart_Mobility,Smart_Environment,Smart_Government,Smart_Economy,Smart_People,Smart_Living,SmartCity_Index
0,0,Oslo,Norway,6480,6512,7516,4565,8618,9090,7138
1,1,Bergen,Norway,7097,6876,7350,4905,8050,9090,7296
2,2,Amsterdam,Netherlands,7540,5558,8528,8095,7098,7280,7311
3,3,Copenhagen,Denmark,7490,7920,8726,5580,5780,7200,7171
4,4,Stockholm,Sweden,6122,7692,8354,4330,6743,7730,6812
...,...,...,...,...,...,...,...,...,...,...
97,97,Riga,Latvia,4152,4584,4616,7380,3745,4330,4712
98,98,Beijing,China,7610,2998,2806,4905,5183,1980,4449
99,99,St Petersburg,Russia,4588,2908,3622,4515,5390,4100,4191
100,100,Calgary,Canada,6675,4052,5946,8022,6424,8657,6678


In [4]:
smart.City.unique()

array(['Oslo', 'Bergen', 'Amsterdam', 'Copenhagen', 'Stockholm',
       'Montreal', 'Vienna', 'Odense', 'Singapore', 'Boston', 'Zurich',
       'Trondheim', 'Vasteras', 'Aalborg', 'Ottawa', 'Washington, DC',
       'Stavanger', 'Los Angeles', 'Helsinki', 'Vancouver', 'Berlin',
       'Toronto', 'New York', 'Chicago', 'Taipei', 'Dusseldorf',
       'Reykjavik', 'Lyon', 'Helsingborg', 'Munich', 'Philadelphia',
       'Hamburg', 'Geneva', 'Luxembourg', 'Aarhus', 'Turku', 'Espoo',
       'Paris', 'Hannover', 'Strasbourg', 'Joensuu', 'Sydney',
       'Frankfurt am Main', 'San Francisco', 'Oulu', 'Cologne', 'Bochum',
       'Stuttgart', 'Hameenlinna', 'London', 'Tampere', 'Adelaide',
       'Melbourne', 'Tokyo', 'Jyvaskyla', 'Perth', 'Vantaa', 'Auckland',
       'Osaka', 'Bordeaux', 'Milan', 'Madrid', 'Daejeon', 'Nantes',
       'Brussels', 'Marseille', 'Leeds', 'Lahti', 'Dublin', 'Abu Dhabi',
       'Verona', 'Seoul', 'Nice', 'Tallinn', 'Dubai', 'Catania',
       'Bayreuth', 'Turin', 'Flore

In [ ]:
smart.Country.unique()

# 3. Request data from APIS

## 3.1 API COUNTRY - CONTINENTS

In [ ]:
import requests

url = "https://referential.p.rapidapi.com/v1/country"

querystring = {"fields":"currency,currency_num_code,currency_code,continent_code,currency,iso_a3,dial_code"}

headers = {
    'x-rapidapi-host': "referential.p.rapidapi.com",
    'x-rapidapi-key': "33e030ac4cmshbfb8d0e76a97e42p1fe169jsn3da51bbbf4d4"
    }

response = requests.request("GET", url, headers=headers, params=querystring)

#print(response.text)

In [ ]:
response.json()

In [ ]:
def decodear(pais):
    for diccionario in response.json():
        if pais == "China":
            return "AS"
        elif pais == diccionario["value"]:
            return diccionario["continent_code"]

In [ ]:
smart["prueba"] = smart.Country.apply(decodear)

In [ ]:
smart.sample(20)

In [ ]:
smart.prueba.unique()

In [ ]:
smart.prueba.fillna(0, inplace=True)

In [ ]:
smart[smart["prueba"] == 0]

In [ ]:
df = json_normalize(response.json())

In [ ]:
# Drop unnecessary columns
df.drop(columns=[
    'iso_a3','currency','currency_code','key', 'currency_num_code'],inplace=True)

In [ ]:
df.sample()

In [ ]:
df["continent_code"].unique()

In [ ]:
df.value.unique()

In [ ]:
#df_smart = smart.merge(df,left_on="Country", right_on="value", suffixes=["_izq", "_dcha"])


In [ ]:
df_smart = df.merge(smart,left_on="value", right_on="Country", suffixes=["_izq", "_dcha"])
#df_smart.sample()

In [ ]:
df_smart

In [ ]:
df_smart["continent_code"].unique()

In [ ]:
df_smart["Continent"] = df_smart["Continent"].str.replace("NA", "North America")
df_smart["Continent"] = df_smart["Continent"].str.replace("OC", "Oceania")
df_smart["Continent"] = df_smart["Continent"].str.replace("EU", "Europe")
df_smart["Continent"] = df_smart["Continent"].str.replace("AS", "Asia")

In [ ]:
df_smart.drop(columns=['value'], inplace = True)

In [ ]:
continent_code_rename = {"continent_code": "Continent"}
df_smart.rename(columns=continent_code_rename, inplace=True)

In [ ]:
df_smart.shape

In [ ]:
df["Continent"].unique()

In [ ]:
df_smart.isnull().sum()

In [ ]:
smart.to_csv("../data/continent_smart.csv")

## 3.1 API LIFE QUALITY

In [ ]:
smart.City.unique()

In [ ]:
# as the API only recives certain names in lowercase, I´m going to change the city names
# into lowercase

In [ ]:
#cities_lower = map(lambda city:city.lower(),smart.City)
print(list(cities_lower))

In [ ]:
df_smart["City"] = df_smart["City"].apply(cf.lowercase)

In [ ]:
"""dict_rename = {
    "dÃ\x83Â¼sseldorf" : "dusseldorf",
    "vã\x83â¤sterã\x83â¥s" : "vasteras", V%C3%A4ster%C3%A5s
    "hã\x83â¤meenlinna" : "hameenlinna",
    "jyvã\x83â¤skylã\x83â¤" : "jyväskylä"
}
df_smart.rename(df_smart["City"] == dict_rename, inplace=True)
df_smart.head()
"""

In [ ]:
df_smart.City.unique()

In [ ]:
cities = list(smart.City.unique())

In [ ]:
scores =[]
for city in cities:
    response = requests.get(f"https://api.teleport.org/api/urban_areas/slug:{city}/scores/").json().["teleport_city_score"]
    scores.append(response)

##### EJEMPLO OSLO

In [ ]:
oslo = "https://api.teleport.org/api/urban_areas/slug:oslo/scores/"

In [ ]:
response_oslo = requests.get(oslo).json()

In [ ]:
response_oslo

In [ ]:
json_normalize(response_oslo)

In [ ]:
response_oslo["teleport_city_score"]

In [ ]:
response_oslo["summary"]

In [ ]:
response_oslo["summary"].strip()